In [1]:
import pandas as pd
#import os
#pip install openpyxl
import numpy as np
#import scipy as sp
import math 

In [2]:
Data=pd.read_excel("DataCS-2018-2021.xlsx",
                   sheet_name=None,
                   usecols="B:P",
                   skipfooter=15,
                   header=7)



#LOADING SHEETS
Data18=Data['2018']
Data19=Data['2019']
Data20=Data['2020']
Data21=Data['2021']

#Data.keys()
#RENAME

# RENAME COLUMNS
new_columns = {0: 'Ma CK', 
               1: 'Ten Cong ty',
               2: 'San',
               3: 'Nhom nganh',
               4: 'Tong Tai san',
               5: 'Tong No phai tra',
               6: 'Von Chu so huu',
               7: 'No ngan han',
               8: 'No dai han',
               9: 'Tai san co dinh',
               10: 'Gia von hang ban',
               11: 'Loi nhuan truoc thue',
               12: 'Chi phi lai vay',
               13: 'Von hoa thi truong',
               14: 'ROA'               
              }

#RENAME and DROP ROWS

Data18=Data18.rename(columns={Data18.columns[i]: new_col for i, new_col in new_columns.items()})
Data19=Data19.rename(columns={Data19.columns[i]: new_col for i, new_col in new_columns.items()})
Data20=Data20.rename(columns={Data20.columns[i]: new_col for i, new_col in new_columns.items()})
Data21=Data21.rename(columns={Data21.columns[i]: new_col for i, new_col in new_columns.items()})

#DROP ROW 1-2
Data18.drop([0,1], axis=0, inplace=True)
Data19.drop([0,1], axis=0, inplace=True)
Data20.drop([0,1], axis=0, inplace=True)
Data21.drop([0,1], axis=0, inplace=True)

# LOC trong San Giao dich HOSE HNX UPCOM
San_GD = ['HOSE','HNX','UPCOM']
Data18 = Data18[Data18['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data19 = Data19[Data19['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data20 = Data20[Data20['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data21 = Data21[Data21['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)



In [3]:
# Tong hop Du lieu tu Nam 2018 - 2021
Data18['Nam']=2018
Data19['Nam']=2019
Data20['Nam']=2020
Data21['Nam']=2021
# Ket hop GDP va COVID
DataGDP_COVID=pd.read_csv("DataGDP_COVID.csv")
DataNganh=pd.read_csv("Index_NhomNganh.csv")

# Data Tong hop
DataAll=pd.concat([Data18, Data19,Data20,Data21]).reset_index(drop=True)
DataAll=DataAll.merge(DataGDP_COVID, how='left', on='Nam')
DataAll=DataAll.merge(DataNganh, how='left', on='Nhom nganh')



In [4]:
#Don Vi la Unit: Million M
DataAll.iloc[:,4:13]=DataAll.iloc[:,4:13]//10**6 # Chia lay lam tron
#Von hoa thi truong, Unit: Ty dong, Billion  
DataAll['Von hoa thi truong']=DataAll['Von hoa thi truong'].apply(np.ceil)  #lam trong so tu nhien


In [6]:
# FILTER : TONG TAI SAN > 100B; Von Hoa thi truong > 100B
#DataAll=DataAll[(DataAll['Tong Tai san'] >= 100*10**3) & (DataAll['Von hoa thi truong'] >= 100 )].reset_index(drop=True)
DataAll

,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Tong No phai tra,Von Chu so huu,No ngan han,No dai han,Tai san co dinh,...,GRWTH,SIZE,EBIT,PROF,LIQD,UNIQ,TANG,STLEV,LTLEV,BLEV
0,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,53624854,22838548,30786305,7683984,15154564,18447261,...,0.006047,17.797523,7713260,0.143837,6.978783,0.154697,0.344006,0.143291,0.282603,0.425895
1,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,64578613,30498935,34079678,15795515,14703420,29203764,...,0.004478,17.983394,8911355,0.137992,4.088415,0.407352,0.45222,0.244594,0.227682,0.472276
2,VNM,VINAMILK,HOSE,Hàng Tiêu dùng L1,37366108,11094739,26271369,10639592,455147,13365353,...,0.00572,17.436275,12103064,0.323905,3.511987,0.748019,0.357687,0.284739,0.012181,0.29692
3,HPG,Hòa Phát,HOSE,Nguyên vật liệu L1,78223007,37600057,40622949,22636149,14963908,12782560,...,0.003428,18.175074,10610934,0.13565,3.455668,0.564612,0.163412,0.28938,0.191298,0.480678
4,SAB,SABECO,HOSE,Hàng Tiêu dùng L1,22366742,6254837,16111905,5925696,329140,4567091,...,0.007244,16.923086,5425684,0.242578,3.774534,1.245797,0.204191,0.264933,0.014716,0.279649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,GDA,Tôn Đông Á,HOSE,Nguyên vật liệu L1,12648113,9196195,3451918,8745065,451129,2834481,...,0.0,16.353019,1709097,0.135127,1.446314,1.776551,0.224103,0.691413,0.035668,0.72708
5280,DHQ,Duyên hải Quảng Ninh,UPCOM,Dịch vụ Tiêu dùng L1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5281,DTJ,ĐT XD Viễn thông Đồng Tháp,UPCOM,Công nghiệp L1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5282,DTN,Diêm Thống Nhất,UPCOM,Hàng Tiêu dùng L1,41094,12358,28735,12358,0,3887,...,0.0,10.623617,2552,0.062102,3.325295,2.605636,0.094588,0.300725,0.0,0.300725


In [5]:
# Tinh toan Nang cao
#GRWTH = MarketCap / Book value of Equity
DataAll['GRWTH']=DataAll['Von hoa thi truong']/DataAll['Von Chu so huu']
# SIZE = natural logarithym LN(Total ASSETS)
DataAll['SIZE'] = np.log(DataAll['Tong Tai san'].astype('float'))
# EBIT = LNTT + Chi phi lai vay => PRO = EBIT/Total Assets ; RISK=Std(EBIT)
DataAll['EBIT']=DataAll['Loi nhuan truoc thue']+abs(DataAll['Chi phi lai vay'])
# Profitability PRO = EBIT/Tong Tai san
DataAll['PROF']=DataAll['EBIT']/DataAll['Tong Tai san']
# LIQD	Tổng tài sản ngắn hạn/ Nợ ngắn hạn
DataAll['LIQD']=DataAll['Tong Tai san']/DataAll['No ngan han']
# UNIQ	Giá vốn hàng bán/Tổng tài sản
DataAll['UNIQ']=abs(DataAll['Gia von hang ban'])/DataAll['Tong Tai san']
# Tính hữu hình	TANG	Tài sản cố định/ Tổng tài sản
DataAll['TANG']=DataAll['Tai san co dinh']/DataAll['Tong Tai san']
#BIEN PHU THUOC: STLEV, LTLEV, BLEV
DataAll['STLEV']=DataAll['No ngan han']/DataAll['Tong Tai san']
DataAll['LTLEV']=DataAll['No dai han']/DataAll['Tong Tai san']
DataAll['BLEV']=DataAll['Tong No phai tra']/DataAll['Tong Tai san']

In [13]:
Output=DataAll.iloc[:,[0,1,2,3,4,13,15,16,17,18,19,20,22,23,24,25,26,27,28]]
Output

,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Von hoa thi truong,Nam,GDP,COVID,INDS,GRWTH,SIZE,PROF,LIQD,UNIQ,TANG,STLEV,LTLEV,BLEV
0,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,53624854,186158,2018,7.17,0,1.0,0.006047,17.797523,0.143837,6.978783,0.154697,0.344006,0.143291,0.282603,0.425895
1,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,64578613,152624,2018,7.17,0,2.0,0.004478,17.983394,0.137992,4.088415,0.407352,0.45222,0.244594,0.227682,0.472276
2,VNM,VINAMILK,HOSE,Hàng Tiêu dùng L1,37366108,150268,2018,7.17,0,2.0,0.00572,17.436275,0.323905,3.511987,0.748019,0.357687,0.284739,0.012181,0.29692
3,HPG,Hòa Phát,HOSE,Nguyên vật liệu L1,78223007,139265,2018,7.17,0,3.0,0.003428,18.175074,0.13565,3.455668,0.564612,0.163412,0.28938,0.191298,0.480678
4,SAB,SABECO,HOSE,Hàng Tiêu dùng L1,22366742,116714,2018,7.17,0,2.0,0.007244,16.923086,0.242578,3.774534,1.245797,0.204191,0.264933,0.014716,0.279649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,GDA,Tôn Đông Á,HOSE,Nguyên vật liệu L1,12648113,0,2021,2.58,1,3.0,0.0,16.353019,0.135127,1.446314,1.776551,0.224103,0.691413,0.035668,0.72708
5280,DHQ,Duyên hải Quảng Ninh,UPCOM,Dịch vụ Tiêu dùng L1,NaN,0,2021,2.58,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5281,DTJ,ĐT XD Viễn thông Đồng Tháp,UPCOM,Công nghiệp L1,NaN,0,2021,2.58,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5282,DTN,Diêm Thống Nhất,UPCOM,Hàng Tiêu dùng L1,41094,0,2021,2.58,1,2.0,0.0,10.623617,0.062102,3.325295,2.605636,0.094588,0.300725,0.0,0.300725


In [14]:
# EXPORT FILE DATA
with pd.ExcelWriter('Report.xlsx') as writer:
    DataAll.to_excel(writer, sheet_name='Data')
    Output.to_excel(writer, sheet_name='Output')